## Salary Prediction for Hitters Dataset with Linear and Non-Linear Models

In this project, we will build several linear and non-linear models on the Hitters dataset. After creating base models, some of them will be tuned to minimize prediction errors in terms of the RMSE metric. To split the dataset for training and test purposes, K-fold Cross-Validation will be used. Linear models used in this project are Linear Regression, Ridge, Lasso, ElasticNet. Non-linear models used in this project are regression implementations of K-Nearest Neighbors (KNN), Classification and Regression Trees (CART), Random Forest (RF), Support Vector Machine (SVM), Gradient Boosting Machine (GBM), eXtreme Gradient Boosting (XGB), and Light-GBM (LGBM).

### Context of Study:

* Understanding Data
* Exploratory Data Analysis
* Data Preparation
* Feature Engineering
* Constructing Base Models (Linear and Non-linear)
* Hyperparameter Tuning for Some Models
* Conclusion

### Understanding Data

Hitters dataset includes Major League Baseball (MLB) data from the seasons 1986 and 1987.
Source: This dataset was taken from the StatLib library which is maintained at Carnegie Mellon University.
A data frame with 322 observations of major league players on the following 20 variables.

#### Variables:
* AtBat: Number of times at bat in 1986
* Hits: Number of hits in 1986
* HmRun: Number of home runs in 1986
* Runs: Number of runs in 1986
* RBI: Number of runs batted in in 1986
* Walks: Number of walks in 1986
* PutOuts: Number of put outs in 1986
* Assists: Number of assists in 1986
* Errors: Number of errors in 1986
* CAtBat: Number of times at bat during his career
* CHits: Number of hits during his career
* CHmRun: Number of home runs during his career
* CRuns: Number of runs during his career
* CRBI: Number of runs batted in during his career
* CWalks: Number of walks during his career
* Years:Number of years in the major leagues
* League: A factor with levels A and N indicating player's league at the end of 1986
* Division: A factor with levels E and W indicating player's division at the end of 1986
* NewLeague: A factor with levels A and N indicating player's league at the beginning of 1987
* Salary: 1987 annual salary on opening day in thousands of dollars


In [1]:
# install required model libraries
#!pip install xgboost
#!pip install lightgbm

In [2]:
# ignore warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_val_score

# linear models
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
# non-linear models
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [3]:
#read the Hitters dataset
data = pd.read_csv("../input/hitters-baseball-data/Hitters.csv")
# copy loaded data into df
df = data.copy()

### Exploratory Data Analysis

In [4]:
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [5]:
# number of rows and columns in data
df.shape

(322, 20)

In [6]:
# descriptive statistics for data
df.describe().T

,count,mean,std,min,25%,50%,75%,max
AtBat,322.0,380.928571,153.404981,16.0,255.25,379.5,512.00,687.0
Hits,322.0,101.024845,46.454741,1.0,64.00,96.0,137.00,238.0
HmRun,322.0,10.770186,8.709037,0.0,4.00,8.0,16.00,40.0
Runs,322.0,50.909938,26.024095,0.0,30.25,48.0,69.00,130.0
RBI,322.0,48.027950,26.166895,0.0,28.00,44.0,64.75,121.0
Walks,322.0,38.742236,21.639327,0.0,22.00,35.0,53.00,105.0
Years,322.0,7.444099,4.926087,1.0,4.00,6.0,11.00,24.0
CAtBat,322.0,2648.683230,2324.205870,19.0,816.75,1928.0,3924.25,14053.0
CHits,322.0,717.571429,654.472627,4.0,209.00,508.0,1059.25,4256.0
CHmRun,322.0,69.490683,86.266061,0.0,14.00,37.5,90.00,548.0


### Data Preparation

In [7]:
# categorical features
cat_cols = [col for col in df.columns if df[col].dtypes == "O"]
# numerical features except for target (Salary)
num_cols = [col for col in df.columns if df[col].dtypes != "O" and col not in "Salary"]

In [8]:
# check outliers in data and replace them with thresholds
q1 = 0.10
q3 = 0.90
for col in num_cols:
    quartile1 = df[col].quantile(q1)
    quartile3 = df[col].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    if df[(df[col] > up_limit) | (df[col] < low_limit)].any(axis=None):
        df.loc[(df[col] < low_limit),col] = low_limit
        df.loc[(df[col] > up_limit), col] = up_limit

In [9]:
# remove outliers for Salary and keep NaN values
salary_up = int(df["Salary"].quantile(q3))
df = df[(df["Salary"] < salary_up) | (df["Salary"].isnull())]
# data shape after removing outliers in Salary
df.shape

(295, 20)

In [10]:
# number of NaN values in each feature
df.isnull().sum()

AtBat         0
Hits          0
HmRun         0
Runs          0
RBI           0
Walks         0
Years         0
CAtBat        0
CHits         0
CHmRun        0
CRuns         0
CRBI          0
CWalks        0
League        0
Division      0
PutOuts       0
Assists       0
Errors        0
Salary       59
NewLeague     0
dtype: int64

In [11]:
# remove rows containing NaN values
df.dropna(inplace=True)
df.shape

(236, 20)

In [12]:
# label encoding of categorical features (League, Division, NewLeague) with two class 
binary_cols = [col for col in df.columns if df[col].dtype not in [int, float] and df[col].nunique() == 2]
for col in binary_cols:
    labelencoder = LabelEncoder()
    df[col] = labelencoder.fit_transform(df[col])

In [13]:
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
1,315,81,7,24,38,39,14,3449,835.0,69.0,321,414,375.0,1,1,632,43,10,475.0,1
2,479,130,18,66,72,76,3,1624,457.0,63.0,224,266,263.0,0,1,880,82,14,480.0,0
3,496,141,20,65,78,37,11,5628,1575.0,225.0,828,838,354.0,1,0,200,11,3,500.0,1
4,321,87,10,39,42,30,2,396,101.0,12.0,48,46,33.0,1,0,805,40,4,91.5,1
5,594,169,4,74,51,35,11,4408,1133.0,19.0,501,336,194.0,0,1,282,421,25,750.0,0


### Feature Engineering

For creating new features, it is important to have domain knowledge about the data we have. For this purpose, I tried to get insights from the glossary of mlb.com the official site of Major League Baseball. I also created some features myself that I think might be useful.

In [14]:
df["New_BattingAverage"] = df["CHits"] / df["CAtBat"]
df["New_TotalBases"] =  ((df["CHits"] * 2) + (4 * df["CHmRun"]))
df["New_SluggingPercentage"] = df["New_TotalBases"] / df["CAtBat"]
df["New_IsolatedPower"] = df["New_SluggingPercentage"] - df["New_BattingAverage"]
df["New_TripleCrown"] = (df["CHmRun"] * 0.4) + (df["CRBI"] * 0.25) + (df["New_BattingAverage"] * 0.35)
df["New_BattingAverageOnBalls"] = (df["CHits"] - df["CHmRun"]) / (df["CAtBat"] - df["CHmRun"])
df["New_RunsCreated"] = df["New_TotalBases"] * (df["CHits"] + df["CWalks"]) / (df["CAtBat"] + df["CWalks"])
df["New_FieldingPercentage"] = 1 - ((df["PutOuts"] + df["Assists"]) / (df["PutOuts"] + df["Assists"] + df["Errors"] + 1))

df["New_CRunsYearsRatio"] = df["CRuns"] / df["Years"]
df['New_PutOutsYears'] = df['PutOuts'] * df['Years']
df["New_RBIWalks"] = df["RBI"] * df["Walks"]
df["New_RBIWalksRatio"] = df["RBI"] / df["Walks"]
df["New_CHmRunCAtBatRatio"] = df["CHmRun"] / df["CAtBat"]

### Constructing Base Models (Linear and Non-linear)

In [15]:
# assign X (input features) and y (target feature)
X = df.drop(["Salary"], axis=1)
y = df["Salary"]

In [16]:
# list feature importances for a regressor model like LGBM
pre_model = LGBMRegressor(random_state=17).fit(X, y)
feature_imp = pd.DataFrame({'Feature': X.columns, 'Value': pre_model.feature_importances_})
feature_imp.sort_values("Value", ascending=False)

,Feature,Value
15,PutOuts,54
30,New_RBIWalksRatio,49
6,Years,47
28,New_PutOutsYears,43
27,New_CRunsYearsRatio,43
31,New_CHmRunCAtBatRatio,37
9,CHmRun,36
24,New_BattingAverageOnBalls,35
12,CWalks,34
17,Errors,31


In [17]:
base_models = [('LR', LinearRegression()), 
               ("Ridge", Ridge(random_state=17)),
               ("Lasso", Lasso(random_state=17)),
               ("ElasticNet", ElasticNet(random_state=17)),
               ('KNN', KNeighborsRegressor()),
               ('CART', DecisionTreeRegressor(random_state=17)),
               ('RF', RandomForestRegressor(random_state=17)),
               ('SVR', SVR()),
               ('GBM', GradientBoostingRegressor(random_state=17)),
               ("XGBoost", XGBRegressor(objective='reg:squarederror', random_state=17)),
               ("LightGBM", LGBMRegressor(random_state=17))]

for name, model in base_models:
    rmse = np.mean(np.sqrt(-cross_val_score(model, X, y, cv=10, scoring="neg_mean_squared_error")))
    print(f"RMSE: {round(rmse, 4)} ({name}) ")

RMSE: 233.2044 (LR) 
RMSE: 228.0199 (Ridge) 
RMSE: 224.5419 (Lasso) 
RMSE: 223.048 (ElasticNet) 
RMSE: 191.639 (KNN) 
RMSE: 210.18 (CART) 
RMSE: 155.6984 (RF) 
RMSE: 269.8491 (SVR) 
RMSE: 164.1558 (GBM) 
RMSE: 176.9497 (XGBoost) 
RMSE: 162.4859 (LightGBM) 


### Hyperparameter Tuning for Some Models

In [18]:
# GridSearchCV parameter space for selected models
knn_params = {"n_neighbors": list(range(1, 31)),
              "algorithm": ["auto", "ball_tree", "kd_tree", "brute"]}

cart_params = {'max_depth': range(1, 20),
               "min_samples_split": range(2, 30)}

rf_params = {"max_depth": [5, 8, 15, None],
             "max_features": [5, 7, "auto"],
             "min_samples_split": [8, 15, 20],
             "n_estimators": [100, 200, 300]}

lightgbm_params = {"learning_rate": [0.01, 0.1, 0.001],
                   "n_estimators": [300, 500, 1500],
                   "colsample_bytree": [0.5, 0.7, 1]}

In [19]:
fast_models = [("KNN", KNeighborsRegressor(), knn_params),
              ("CART", DecisionTreeRegressor(random_state=17), cart_params),
              ("RF", RandomForestRegressor(random_state=17), rf_params),
              ("LightGBM", LGBMRegressor(random_state=17), lightgbm_params)]

In [20]:
best_models = {}

for name, model, params in fast_models:
    rmse = np.mean(np.sqrt(-cross_val_score(model, X, y, cv=10, scoring="neg_mean_squared_error")))
    print(f"RMSE (Before): {round(rmse, 4)} ({name}) ")
    gs_best = GridSearchCV(model, params, cv=3, n_jobs=-1, verbose=False).fit(X, y)
    final_model = model.set_params(**gs_best.best_params_)
    rmse = np.mean(np.sqrt(-cross_val_score(final_model, X, y, cv=10, scoring="neg_mean_squared_error")))
    print(f"RMSE (After): {round(rmse, 4)} ({name}) ")
    print(f"{name} best params: {gs_best.best_params_}", end="\n\n")
    best_models[name] = final_model

RMSE (Before): 191.639 (KNN) 
RMSE (After): 180.8777 (KNN) 
KNN best params: {'algorithm': 'auto', 'n_neighbors': 21}

RMSE (Before): 210.18 (CART) 
RMSE (After): 177.8424 (CART) 
CART best params: {'max_depth': 7, 'min_samples_split': 18}

RMSE (Before): 155.6984 (RF) 
RMSE (After): 154.9561 (RF) 
RF best params: {'max_depth': 15, 'max_features': 'auto', 'min_samples_split': 8, 'n_estimators': 200}

RMSE (Before): 162.4859 (LightGBM) 
RMSE (After): 155.7668 (LightGBM) 
LightGBM best params: {'colsample_bytree': 0.5, 'learning_rate': 0.01, 'n_estimators': 300}



### Conclusion

In conclusion, the best model seems like **RF** with a **154.9561** score of root-mean-squared error (RSME). RMSE is a frequently used measure of the differences between values (sample or population values) predicted by a model or an estimator and the values observed. 

Considering the feature importances, it is seen that the newly created features are effective in the success of the model, although it varies according to the model used.

* After setting q1=0.05, q3=0.95, and removing outliers in "Salary", 249 rows left and achieved 185.6107 RSME score with tuned RF.
* After setting q1=0.10, q3=0.90, and removing outliers in "Salary", 236 rows left and achieved 154.9561 RSME score with tuned RF.
* After setting q1=0.20, q3=0.80, and removing outliers in "Salary", 210 rows left and achieved 126.4241 RSME score with tuned RF.
* After setting q1=0.25, q3=0.75, and removing outliers in "Salary", 193 rows left and achieved 118.4944 RSME score with tuned RF.